In [8]:
#from MuZeroParallel import MuZero as MuZeroP

from MuZero import MuZero

from GridWorldOption import GridWorldOption


from GridWorld import GridWorld

from mcts.MCTS import MCTS
import numpy as np
import matplotlib.pyplot as plt

from random import choice
from copy import deepcopy
import time

import pickle
import pandas as pd

from matplotlib.pyplot import figure
import itertools
from tqdm import tqdm


In [9]:
MAP_NAME = './maps/door.map'
with open(MAP_NAME) as f:
    the_map = f.read()

In [10]:
def get_g(full_options):
    env = GridWorld(the_map)

    G_model = {}
    for position in tqdm(env.possible_positions):
        for option in full_options:
            if option.is_valid(position):

                test_env = GridWorld(the_map)
                test_env.cur_x = position[0]
                test_env.cur_y = position[1]
                rewards = []
                while True:
                    action, should_break = option.get_action(test_env)

                    if action == -1:
                        break

                    next_position, r, _ = test_env.step(action)
                    rewards.append(r)

                    if should_break:
                        break

                G_model[(position, option)] = (next_position, rewards)
    return G_model


In [11]:
def get_v():
    V_model = {}

    env = GridWorld(the_map)
    the_opt  = GridWorldOption((env.goal_x, env.goal_y),   {'all'}, 10)

    for position in tqdm(env.possible_positions):
        env_test = GridWorld(the_map)

        the_opt  = GridWorldOption((env_test.goal_x, env_test.goal_y),   {'all'}, 10)


        env_test.cur_x = position[0]
        env_test.cur_y = position[1]

        state = position
        rewards = []
        done = False
        while True:

            action, _ = the_opt.get_action(env_test)

            if action == -1 or done:
                break

            #print(action)
            state, r, done = env_test.step(action)
            #print(state)
            rewards.append(r)

        G = 0
        counter = 0
        for r in rewards:
            G += (0.99 ** counter) * r
            counter += 1

        #print(len(rewards))
        #print(G)

        V_model[position] = G
    return V_model

In [12]:
MAP_NAME = './maps/door.map'
with open(MAP_NAME) as f:
    the_map = f.read()

    
first_room_pos = [(i,j) for i in range(6) for j in range(8)]
second_room_pos = [(i,j) for i in range(6) for j in range(9, 15)]
third_room_pos = [(i,j) for i in range(7,13) for j in range(8)]
fourth_room_pos = [(i,j) for i in range(7,13) for j in range(9,15)]

options = [
    # primitives, (0,0) is meaningless
    GridWorldOption((0, 0),   {'all'}, 0, 0),
    GridWorldOption((0, 0),   {'all'}, 1, 1),
    GridWorldOption((0, 0),   {'all'}, 2, 2),
    GridWorldOption((0, 0),   {'all'}, 3, 3),
    GridWorldOption((3,8),  set(first_room_pos + second_room_pos + [(6,3)] + [(6,13)]), 4),
    GridWorldOption((6,3),  set(first_room_pos + third_room_pos + [(3,8)] + [(11,8)]), 5),
    GridWorldOption((6,13), set(second_room_pos + fourth_room_pos + [(3,8)] + [(11,8)]), 6),
    GridWorldOption((11,8), set(third_room_pos + fourth_room_pos + [(6,3)] + [(6,13)]), 7),
]

G_model = get_g(options)
V_model = get_v()

env = GridWorld(the_map)
mu_debug = MuZero(env, options)


s_next = mu_debug.env.reset()


s0 = s_next
mu_debug.f.v_table = V_model
mu_debug.g.model_table = G_model

100%|███████████████████████████████████████| 172/172 [00:00<00:00, 2012.81it/s]


In [13]:
mcts = MCTS(s0, mu_debug.f, mu_debug.g, mu_debug.options)


mcts.run_sim(10000)

array([0.0427, 0.1017, 0.0992, 0.0424, 0.4594, 0.2546, 0.    , 0.    ])

In [14]:
mcts.info()

Expanded:  True
Leaf:  False
hidden state (1, 1)
Option 0
N(s, o) 427
Q(s, o) -25.419632052539367
P(s, o) 0.16666666666666666
Q'(s, o):  0.0006470131661985059
UCT exploration:  0.2336448598130841
Prior regulation:  1.661384186457386
U(s, o) 0.06534265905628361
Option 1
N(s, o) 1017
Q(s, o) -24.54241175492071
P(s, o) 0.16666666666666666
Q'(s, o):  0.03806342340134884
UCT exploration:  0.09823182711198428
Prior regulation:  1.661384186457386
U(s, o) 0.06526355742979327
Option 2
N(s, o) 992
Q(s, o) -24.562226891805107
P(s, o) 0.16666666666666666
Q'(s, o):  0.0372182408485227
UCT exploration:  0.10070493454179255
Prior regulation:  1.661384186457386
U(s, o) 0.06510317180618275
Option 3
N(s, o) 424
Q(s, o) -25.429585948862027
P(s, o) 0.16666666666666666
Q'(s, o):  0.00022244584911995609
UCT exploration:  0.23529411764705882
Prior regulation:  1.661384186457386
U(s, o) 0.0653747668866645
Option 4
N(s, o) 4594
Q(s, o) -24.042272799249297
P(s, o) 0.16666666666666666
Q'(s, o):  0.05939604006649